In [1]:
from datasets import load_dataset

# Load training split
dataset = load_dataset("openpecha/OCR-Google_Books")

# Example features
print(dataset)
# {
#     'filename': 'image_001.jpg',
#     'label': '༄༅། །རྒྱལ་པོ་ ལ་ གཏམ་ གྱི་ བྱུང་ བ་ བཤད་ པ།',
#     'url': 'https://...',
#     'BDORC_work_id': 'W...',
#     'char_len': 25,
#     'script': 'ScriptTibt',
#     'print_method': 'Modern'
# }

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'url', 'image'],
        num_rows: 601152
    })
    eval: Dataset({
        features: ['id', 'label', 'url', 'image'],
        num_rows: 75136
    })
    test: Dataset({
        features: ['id', 'label', 'url', 'image'],
        num_rows: 75168
    })
})


In [2]:
dataset['train'][0]

{'id': 'I1KG1163750042_0025',
 'label': 'ཡིན་པས་ཆབ་སྲིད་དང་འབྲེལ་བ་བྱུང་བ་ཙམ་ལ་ངོ་མཚར་དགོས་དོན་གང་ ',
 'url': 'https://s3.amazonaws.com/monlam.ai.ocr/OCR/training_images/I1KG1163750042_0025.jpg',
 'image': None}

## Remove Unused Columns

In [3]:
#dataset = dataset.remove_columns(['BDRC_work_id', 'char_len', 'script', 'print_method']).rename_column('filename', 'id')
#dataset['train'][0]

## Add Placeholder Image Column

In [4]:
"""from datasets import Image as ImageFeature

def add_placeholder_image(example):
    #Add a None placeholder for the image column

    example['image'] = None
    return example

# Add the image column with placeholders
print("Adding placeholder image column...")
dataset = dataset.map(
    add_placeholder_image,
    desc="Adding image column"
)

dataset = dataset.cast_column('image', ImageFeature())

dataset['train'][0]"""

'from datasets import Image as ImageFeature\n\ndef add_placeholder_image(example):\n    #Add a None placeholder for the image column\n\n    example[\'image\'] = None\n    return example\n\n# Add the image column with placeholders\nprint("Adding placeholder image column...")\ndataset = dataset.map(\n    add_placeholder_image,\n    desc="Adding image column"\n)\n\ndataset = dataset.cast_column(\'image\', ImageFeature())\n\ndataset[\'train\'][0]'

## Download and Append Images

In [5]:
import requests
from PIL import Image
import io
from datasets import Dataset, Image as ImageFeature
import pandas as pd

def download_image(example):
    """
    Function to be used with dataset.map() to download images for each example
    """
    try:
        # Download the image
        response = requests.get(example['url'])
        response.raise_for_status()
        
        # Open image and convert to PIL Image object
        # image = io.BytesIO(response.content)
        example['image'] = response.content
    
    except:
        example['image'] = None
    
    return example

In [6]:
eval = dataset['eval'].map(download_image, num_proc=4, batch_size=8)

Map (num_proc=4):   0%|          | 0/75136 [00:00<?, ? examples/s]

In [7]:
from datasets import DatasetDict

dd = DatasetDict()

dd['train'] = dataset['train']
dd['eval'] = eval
dd['test'] = dataset['test']

In [8]:
dd.save_to_disk('ocr-google-books-ds')

Saving the dataset (0/1 shards):   0%|          | 0/601152 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/75136 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/75168 [00:00<?, ? examples/s]

In [11]:
test = dataset['test'].map(download_image, num_proc=4, batch_size=8)

Map (num_proc=4):  50%|#####     | 37584/75168 [00:00<?, ? examples/s]

TimeoutError: 

In [ ]:
from datasets import DatasetDict

dd = DatasetDict()

dd['train'] = dataset['train']
dd['eval'] = eval
dd['test'] = test

In [ ]:
dd.save_to_disk('ocr-google-books-ds')

In [ ]:
train = dataset['train'].map(download_image, num_proc=4, batch_size=8)

In [ ]:
from datasets import DatasetDict

dd = DatasetDict()

dd['train'] = train
dd['eval'] = eval
dd['test'] = test

In [ ]:
dd.save_to_disk('ocr-google-books-ds')

In [ ]:
dd.push_to_hub("openpecha/OCR-Google_Books")

In [ ]:
#dataset.push_to_hub("openpecha/OCR-Google_Books")